# Fine-tuning com Stable Diffusion








## Instalação e configuração

In [1]:
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/examples/dreambooth/train_dreambooth.py
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/scripts/convert_diffusers_to_original_stable_diffusion.py
%pip install -qq git+https://github.com/ShivamShrirao/diffusers
%pip install -q -U --pre triton==2.0.0
%pip install -q accelerate transformers ftfy bitsandbytes==0.35.0 gradio natsort safetensors xformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.6/211.6 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 88.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━

## Carregamento do modelo


> Temos de uso

* https://huggingface.co/spaces/CompVis/stable-diffusion-license


In [2]:
model_sd = "runwayml/stable-diffusion-v1-5"
output_dir = "/content/stable_diffusion_weights/zwx"

In [3]:
print(f"Diretório: {output_dir}")
!mkdir -p $output_dir

Diretório: /content/stable_diffusion_weights/zwx


## Treinamento

Lembrando que para realizar o treinamento com o Dreambooth precisamos do seguinte:
1. identificador único
2. nome de classe
3. imagens do sujeito a ser inserido

O primeiro já temos, fizemos na etapa anterior.

Para nossa implementação, vamos pegar a imagem de um rosto como exemplo. O identificador único pode ser zwx e a classe é person.

Precisamos construir o prompt da instância (instance prompt)
a photo of [identificador único] [nome da classe]

e um prompt de classe (class prompt)
> a photo of [nome da classe]

O prompt da instância nesse exemplo será:
> a photo of zwx person

Como o sujeito é uma pessoa, o prompt da classe será

> a photo of a person

In [4]:
concepts_list = [
    {
        "instance_prompt": "zwx",
        "class_prompt": "photo of a person",
        "instance_data_dir": "/content/data/zwx",
        "class_data_dir": "/content/data/person"
    }
]

In [5]:
import json
import os
import random

for c in concepts_list:
  os.makedirs(c["instance_data_dir"], exist_ok=True)

with open("concepts_list.json", "w") as f:
  json.dump(concepts_list, f, indent=4)

**Fazer o upload das imagens de treinamento na pasta /data/zwx**

### Parâmetros

In [6]:
num_imgs = 16
num_class_images = num_imgs * 12
max_num_steps = num_imgs * 80
learning_rate = 1e-6 # 0.0000001
lr_warmup_steps = int(max_num_steps / 10)
print(num_imgs, num_class_images, max_num_steps, learning_rate, lr_warmup_steps)

16 192 1280 1e-06 128


In [7]:
!python3 train_dreambooth.py \
  --pretrained_model_name_or_path=$model_sd \
  --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-mse" \
  --output_dir=$output_dir \
  --revision="fp16" \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --seed=777 \
  --resolution=512 \
  --train_batch_size=1 \
  --train_text_encoder \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --learning_rate=$learning_rate \
  --lr_scheduler="constant" \
  --lr_warmup_steps=80 \
  --num_class_images=$num_class_images \
  --sample_batch_size=4 \
  --max_train_steps=$max_num_steps \
  --save_interval=10000 \
  --save_sample_prompt="zwx" \
  --concepts_list="concepts_list.json"

2023-10-11 23:57:04.897332: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
safety_checker/model.safetensors not found
Fetching 15 files:   0% 0/15 [00:00<?, ?it/s]

Fetching 15 files:   7% 1/15 [00:00<00:10,  1.28it/s]









































































































































































































Fetching 15 files:  27% 4/15 [00:07<00:22,  2.05s/it]









































































































Fetching 15 files: 100% 15/15 [00:12<00:00,  1.23it/s]
/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
You have disabled the safety checker for <class 'diffus

In [8]:
from natsort import natsorted
from glob import glob
import os

weights_dir = natsorted(glob(output_dir + os.sep + "*"))[-1]
print(f"Diretório com os pesos: {weights_dir}")

Diretório com os pesos: /content/stable_diffusion_weights/zwx/1280


In [9]:
import os
import matplotlib.pyplot as plt
from PIL import Image

def grid_img(imgs, rows=1, cols=3, scale=1):
  assert len(imgs) == rows * cols

  w, h = imgs[0].size
  w, h = int(w*scale), int(h*scale)

  grid = Image.new('RGB', size=(cols*w, rows*h))
  grid_w, grid_h = grid.size

  for i, img in enumerate(imgs):
      img = img.resize((w,h), Image.LANCZOS)
      grid.paste(img, box=(i%cols*w, i//cols*h))
  return grid

In [10]:
weights_folder = output_dir
folders = sorted([f for f in os.listdir(weights_folder) if f != "0"], key = lambda x: int(x))
#print(folders)

imgs_test = []

for imgs, folder in enumerate(folders):
  #print(folder)
  folder_path = os.path.join(weights_folder, folder)
  image_folder = os.path.join(folder_path, "samples")
  images = [f for f in os.listdir(image_folder)]

  for i in images:
    img_path = os.path.join(image_folder, i)
    r = Image.open(img_path)
    imgs_test.append(r)

grid_img(imgs_test, rows=1, cols=4, scale=1)

Output hidden; open in https://colab.research.google.com to view.

## Converte os pesos para ckpt (checkpoint)

In [11]:
ckpt_path = weights_dir + "/model.ckpt"

half_arg = "--half"

!python convert_diffusers_to_original_stable_diffusion.py --model_path $weights_dir  --checkpoint_path $ckpt_path $half_arg
print(f"Converted to ckpt and saved in {ckpt_path}")

Reshaping encoder.mid.attn_1.q.weight for SD format
Reshaping encoder.mid.attn_1.k.weight for SD format
Reshaping encoder.mid.attn_1.v.weight for SD format
Reshaping encoder.mid.attn_1.proj_out.weight for SD format
Reshaping decoder.mid.attn_1.q.weight for SD format
Reshaping decoder.mid.attn_1.k.weight for SD format
Reshaping decoder.mid.attn_1.v.weight for SD format
Reshaping decoder.mid.attn_1.proj_out.weight for SD format
Converted to ckpt and saved in /content/stable_diffusion_weights/zwx/1280/model.ckpt


## Inferência do modelo

In [12]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler
from IPython.display import display

In [13]:
model_path = weights_dir
print(model_path)

/content/stable_diffusion_weights/zwx/1280


In [14]:
pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16).to('cuda')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/h

In [15]:
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
pipe.enable_xformers_memory_efficient_attention()

In [16]:
seed = 777

## Geração das imagens

In [29]:
prompt = "I request the creation of an image of a human face identified as 'zwx', with an emphasis on reality and well-balanced proportions. I look for natural facial features, such as expressive eyes with a visible pupil and iris, defined eyebrows, a nose and mouth in harmonious proportions. The skin must be represented with a natural texture, with realistic skin tones, including subtle nuances and shading.The face must be positioned frontally, with a symmetrical and neutral facial expression, without wrinkles or prominent expression marks. Image quality must be of a high standard, with high resolution, sharp details and realistic colors. Please avoid any distortions, excessive shadows or elements that make the face unrecognizable, such as facial deformities, disproportionate ears, oversized head and cheeks, crossed eyes, deformed hands or dark compositions. Make sure that the general anatomy of the face is that of a human being and that there are no elements that appear inappropriate or disproportionate, such as a forehead that is too wide, an excessively pronounced chin, or excessively thick lips. Please keep the image true to reality, avoiding any exaggerated features, caricatures or unnatural elements. This is a representation of an ordinary, authentic human face."
negative_prompt = "Facial deformations (deformed eyes, closed gaze, distorted face), poor image quality, disproportionate ears, large head and cheeks, crossed eyes, deformed hands, obscure composition, inappropriate anatomy, disfigured appearance, poorly drawn face, poorly drawn hands, poorly drawn feet, blurry image, low resolution, out of frame, signature or watermark."

num_samples = 5
guidance_scale = 7.5
num_inference_steps = 30
height = 512
width = 512

seed = random.randint(0, 2147483647) # gera um valor aleatório
print("Seed: {}".format(str(seed)))
generator = torch.Generator(device='cuda').manual_seed(seed)

with autocast("cuda"), torch.inference_mode():
    imgs = pipe(
        prompt,
        negative_prompt=negative_prompt,
        height=height, width=width,
        num_images_per_prompt=num_samples,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=generator
    ).images

for img in imgs:
    display(img)

Output hidden; open in https://colab.research.google.com to view.

### Mais exemplos de prompts

* in the forest, in cairo, in cairo desert,  in a western scene, in star wars, in mountain fuji, in the snow, etc.

> E combinações mais completas de prompt para você testar:

* `photo of zwx person, closeup, mountain fuji in the background, natural lighting `

* `digital painting of zwx in the snow, realistic, hd, vivid, sunset`

* `watercolor painting of zwx person, realistic, blue and orange tones `

* `digital painting of zwx person, hyperrealistic, fantasy, Surrealist, painted by Alphonse Mucha`

* `painting of zwx person in star wars, realistic, 4k ultra hd, blue and red tones`

* `photo of zwx person, in an armor, realistic, visible face, colored, detailed face, ultra detailed, natural lighting`

* `photo of zwx person, cyberpunk, vivid, realistic, 4k ultra hd`

* `anime painting of zwx person, chill day, by tim okamura, noah bradley, trending on artstation  `

### Testando vários prompts de uma vez

In [ ]:
prompt = ["photo of zwx person, closeup, mountain fuji in the background, natural lighting",
          "photo of zwx person in the desert, closeup, pyramids in the background, natural lighting, frontal face",
          "photo of zwx person in the forest, natural lighting, frontal face",
          "photo of zwx person as an astronaut, natural lighting, frontal face, closeup, starry sky in the background",
          "face portrait of zwx in the snow, realistic, hd, vivid, sunset"]

negative_prompt = ["bad anatomy, ugly, deformed, desfigured, distorted face, poorly drawn hands, poorly drawn face, poorly drawn feet, blurry, low quality, low definition, lowres, out of frame, out of image, cropped, cut off, signature, watermark" ] * len(prompt)
num_samples = 1
guidance_scale = 7.5
num_inference_steps = 30
height = 512
width = 512

seed = random.randint(0, 2147483647) # gera um valor aleatório
print("Seed: {}".format(str(seed)))
generator = torch.Generator(device='cuda').manual_seed(seed)

with autocast("cuda"), torch.inference_mode():
    imgs = pipe(
        prompt,
        negative_prompt=negative_prompt,
        height=height, width=width,
        num_images_per_prompt=num_samples,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=generator
    ).images

for img in imgs:
    display(img)

Output hidden; open in https://colab.research.google.com to view.

## Salvando os resultados

In [ ]:
!mkdir resultados

In [ ]:
for i, img in enumerate(imgs):
  img.save('resultados/resultado_{}.png'.format(i+1))

> Mais informações sobre essa implementação do Dreambooth:
https://github.com/ShivamShrirao/diffusers/tree/main/examples/dreambooth